In [21]:
# 新しいノートブックで:
import lightgbm as lgb
import joblib
import json
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np


In [3]:
cd store-sales-time-series-forecasting

c:\Users\naosa\Desktop\kaggle\test\store-sales-time-series-forecasting


c:\Users\naosa\Desktop\kaggle\test\myenv\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [4]:
# または、joblibで保存したモデルを読み込む場合
loaded_model = joblib.load('lightgbm_model.joblib')
# LabelEncoderの読み込み
loaded_label_encoders = joblib.load('label_encoders.joblib')

In [5]:
# 特徴量名の読み込み
with open('feature_names.json', 'r') as f:
    feature_names = json.load(f)

In [12]:
# 新しいデータに対して予測を行う
# 注意: 新しいデータは学習時と同じ前処理が必要です
x = pd.read_csv("test.csv") 
x.head()


,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0


In [13]:
stores = pd.read_csv("stores.csv") 
stores.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [20]:
result = x["id"]
result

0        3000888
1        3000889
2        3000890
3        3000891
4        3000892
          ...   
28507    3029395
28508    3029396
28509    3029397
28510    3029398
28511    3029399
Name: id, Length: 28512, dtype: int64

In [14]:
# store_nbrをキーに左結合
merged_x = pd.merge(x, stores, on='store_nbr', how='left')
merged_x = merged_x.drop(['date','id'], axis=1)
merged_x.head()


,store_nbr,family,onpromotion,city,state,type,cluster
0,1,AUTOMOTIVE,0,Quito,Pichincha,D,13
1,1,BABY CARE,0,Quito,Pichincha,D,13
2,1,BEAUTY,2,Quito,Pichincha,D,13
3,1,BEVERAGES,20,Quito,Pichincha,D,13
4,1,BOOKS,0,Quito,Pichincha,D,13


In [17]:
cat_columns = ['family', 'city', 'state', 'type']

# LabelEncoderを使用してエンコーディング
label_encoders = {}
for col in cat_columns:
    le = LabelEncoder()
    merged_x[col] = le.fit_transform(merged_x[col].astype(str))
    label_encoders[col] = le

merged_x

,store_nbr,family,onpromotion,city,state,type,cluster
0,1,0,0,18,12,3,13
1,1,1,0,18,12,3,13
2,1,2,2,18,12,3,13
3,1,3,20,18,12,3,13
4,1,4,0,18,12,3,13
...,...,...,...,...,...,...,...
28507,9,28,1,18,12,1,6
28508,9,29,0,18,12,1,6
28509,9,30,1,18,12,1,6
28510,9,31,9,18,12,1,6


In [18]:
#予測
predictions = loaded_model.predict(merged_x)
predictions

array([  27.42015403,   27.42015403,  130.60493415, ..., 1070.61649659,
        492.91597725,   42.90471406])

In [22]:
# 長さの確認
print(f"result の長さ: {len(result)}")
print(f"predictions の長さ: {len(predictions)}")


result の長さ: 28512
predictions の長さ: 28512


In [26]:
# 新しいDataFrameの作成
df_merged = pd.DataFrame({
    'id': result,
    'sales': predictions
})

In [27]:
print(df_merged.head())
print(f"マージされたDataFrameの形状: {df_merged.shape}")


        id        sales
0  3000888    27.420154
1  3000889    27.420154
2  3000890   130.604934
3  3000891  2765.978146
4  3000892    75.428169
マージされたDataFrameの形状: (28512, 2)


In [28]:
#csv保存
df_merged.to_csv('submission.csv', index=False)

#提出完了
評価結果(RMSLE)：1.79358
ローカル評価：2.3376
順位：526/601
